![scrna2/6](https://img.shields.io/badge/scrna2/6-lightgrey)
[![Jupyter Notebook](https://img.shields.io/badge/Source%20on%20GitHub-orange)](https://github.com/laminlabs/lamin-usecases/blob/main/docs/scrna-append.ipynb)
[![lamindata](https://img.shields.io/badge/Source%20%26%20report%20on%20LaminHub-mediumseagreen)](https://lamin.ai/laminlabs/lamindata/transform/ManDYgmftZ8C0003)

# Standardize and append a dataset

Here, we'll learn 
- how to standardize a less well curated dataset
- how to append it to the growing versioned collection

In [ ]:
import lamindb as ln
import bionty as bt

ln.track("ManDYgmftZ8C")

Let's now consider a less-well curated dataset:

In [ ]:
adata = ln.core.datasets.anndata_pbmc68k_reduced()
# we don't trust the cell type annotation in this dataset
adata.obs.rename(columns={"cell_type": "cell_type_untrusted"}, inplace=True)
adata

Create a curator:

In [ ]:
curator = ln.Curator.from_anndata(
    adata,
    var_index=bt.Gene.symbol,
    categoricals={"cell_type_untrusted": bt.CellType.name},
    organism="human",
)
curator.validate()

## Standardize & validate genes ![](https://img.shields.io/badge/Validate-10b981) 

Let's convert Gene symbols to Ensembl ids via {meth}`~docs:lamindb.models.CanCurate.standardize`. Note that this is a non-unique mapping and the first match is kept because the `keep` parameter in `.standardize()` defaults to `"first"`:

In [ ]:
adata.var["ensembl_gene_id"] = bt.Gene.standardize(
    adata.var.index,
    field=bt.Gene.symbol,
    return_field=bt.Gene.ensembl_gene_id,
    organism="human",
)
# use ensembl_gene_id as the index
adata.var.index.name = "symbol"
adata.var = adata.var.reset_index().set_index("ensembl_gene_id")

# we only want to save data with validated genes
validated = bt.Gene.validate(adata.var.index, bt.Gene.ensembl_gene_id, mute=True)
adata_validated = adata[:, validated].copy()

Here, we'll use `.raw`:

In [ ]:
adata_validated.raw = adata.raw[:, validated].to_adata()
adata_validated.raw.var.index = adata_validated.var.index

In [ ]:
curator = ln.Curator.from_anndata(
    adata_validated,
    var_index=bt.Gene.ensembl_gene_id,
    categoricals={"cell_type_untrusted": bt.CellType.name},
    organism="human",
)

In [ ]:
curator.validate()

## Standardize & validate cell types ![](https://img.shields.io/badge/Validate-10b981) 

None of the cell type names are valid.
We'll now look up the non-validated cell types using the values of the public ontology and create a mapping.

In [ ]:
curator.non_validated["cell_type_untrusted"]

In [ ]:
ct_public_lo = bt.CellType.public().lookup()
name_mapping = {
    "Dendritic cells": ct_public_lo.dendritic_cell.name,
    "CD19+ B": ct_public_lo.b_cell_cd19_positive.name,
    "CD4+/CD45RO+ Memory": ct_public_lo.effector_memory_cd45ra_positive_alpha_beta_t_cell_terminally_differentiated.name,
    "CD8+ Cytotoxic T": ct_public_lo.cd8_positive_alpha_beta_cytotoxic_t_cell.name,
    "CD4+/CD25 T Reg": ct_public_lo.cd4_positive_cd25_positive_alpha_beta_regulatory_t_cell.name,
    "CD14+ Monocytes": ct_public_lo.cd14_positive_monocyte.name,
    "CD56+ NK": ct_public_lo.cd56_positive_cd161_positive_immature_natural_killer_cell_human.name,
    "CD8+/CD45RA+ Naive Cytotoxic": ct_public_lo.cd8_positive_alpha_beta_memory_t_cell_cd45ro_positive.name,
    "CD34+": ct_public_lo.cd34_positive_cd56_positive_cd117_positive_common_innate_lymphoid_precursor_human.name,
}

We can now standardize cell type names using the lookup-based mapper:

In [ ]:
adata_validated.obs["cell_type_untrusted_original"] = adata_validated.obs[
    "cell_type_untrusted"
]  # copy the original annotations
adata_validated.obs["cell_type_untrusted"] = adata_validated.obs[
    "cell_type_untrusted_original"
].map(name_mapping)

Now, all cell types are validated:

In [ ]:
curator.validate()

### Register ![](https://img.shields.io/badge/Register-10b981) 

In [ ]:
artifact = curator.save_artifact(description="10x reference adata")

In [ ]:
artifact.view_lineage()

In [ ]:
artifact.describe()

## Re-curate

We review the dataset and find all annotations trustworthy up there being a `'CD38-positive naive B cell'`.

Inspecting the `name_mapping` in detail tells us `'CD8+/CD45RA+ Naive Cytotoxic'` was erroneously mapped on a B cell.

Let us correct this and create a `'cell_type'` feature that we can now trust.

In [ ]:
name_mapping["CD38-positive naive B cell"] = "cytotoxic T cell"

In [ ]:
adata_validated.obs["cell_type"] = adata_validated.obs[
    "cell_type_untrusted_original"
].map(name_mapping)
adata_validated.obs["cell_type"].unique()

In [ ]:
artifact_trusted = ln.Curator.from_anndata(
    adata_validated,
    var_index=bt.Gene.ensembl_gene_id,
    categoricals={
        "cell_type": bt.CellType.name,
        "cell_type_untrusted": bt.CellType.name,
    },
    organism="human",
).save_artifact(
    description="10x reference adata, trusted cell type annotation",
    revises=artifact,
)

In [ ]:
artifact_trusted.describe()

## Append the dataset to the collection

Query the previous collection:

In [ ]:
collection_v1 = ln.Collection.get(key="scrna/collection1")

Create a new version of the collection by sharding it across the new `artifact` and the artifact underlying version 1 of the collection:

In [ ]:
collection_v2 = collection_v1.append(artifact_trusted).save()

If you want, you can label the collection's version by setting `.version`.

In [ ]:
collection_v2.version = "2"
collection_v2.save()

Version 2 of the collection covers significantly more conditions.

In [ ]:
collection_v2.describe()

View data lineage:

In [ ]:
collection_v2.view_lineage()